In [1]:
from main_script import main
main()

[(1, ['user1', 'League of Legends', 'Bayonetta', 'Fire Emblem', 'Uncharted', 'Animal Crossing', 'Far Cry', 'Portal', 'Dragon Age', 'Grand Theft Auto V', 'Red Dead Redemption 2', 'Super Mario Bros', 'Metal Gear Solid', 'Ghost of Tsushima', 'World of Warcraft', 'Dragon Age']), (2, ['user2', 'Diablo', 'League of Legends', 'Silent Hill', 'Sekiro', 'The Legend of Zelda', 'Mass Effect', 'Battlefield', 'Half-Life', 'Red Dead Redemption 2', 'Metroid', 'The Last of Us', 'Spider-Man', 'The Elder Scrolls V: Skyrim', 'Fortnite', 'Resident Evil', 'Bioshock', 'The Legend of Zelda']), (3, ['user3', 'Half-Life', 'Metal Gear Solid', 'Cyberpunk 2077', 'Call of Duty', 'Persona 5', 'Bloodborne', 'Diablo', 'Fortnite', 'The Legend of Zelda', 'Dark Souls', 'Gears of War', 'Spider-Man', 'God of War', 'Dragon Age', 'World of Warcraft', 'The Last of Us', 'Splatoon', 'Battlefield', 'Spider-Man', 'The Legend of Zelda', 'The Witcher 3']), (4, ['user4', 'Fallout', 'Elden Ring', 'Bioshock', 'Bayonetta', 'Resident Ev

For testing: Illustrate user game sets: 9 users, complicated Venn diagram.

In [1]:
# runtime: less than 3 minutes

import csv
from hash_table import HashTable
from random import SystemRandom
# # from matplotlib_venn import venn2, venn3
# from venn import venn
import matplotlib.pyplot as plt
import sys

sys.setrecursionlimit(10000)

# class OrderedSet(list):
#     def __iadd__(self, value):
#         return super().__iadd__(value) if self.count(value) == 0 else None


class ExperimentalHashTable(HashTable):  # this doesn't really need to be hash table
    # def __init__(self, size, collision_avoidance: str = 'separate chaining', initial_data: list[any] = []) -> None:
    def rand_hash_probing(self, hash1: int, probe: int, hash_size: int) -> int:
        hash2 = self.rand_hash(probe)
        return (hash1 + hash2) % hash_size
    
    def __init__(self, size: int, collision_avoidance: str, initial_data: list[any] =[]):
        super().__init__(size, collision_avoidance)
        if collision_avoidance.lower().count('rand') > 0:
            self.probe_function = self.rand_hash_probing
        self.rand_key_dict = HashTable(1)
        self.rand_key_set = HashTable(1)
    
    def rand_hash(self, key: int) -> int:
        hash_bool = True
        count = 0
        while hash_bool:
            if not self.rand_key_dict.contains(key):
                rand_16bits = SystemRandom().getrandbits(16)
                # print(rand_16bits)
                if not self.rand_key_set.contains(rand_16bits):
                    self.rand_key_set.insert(rand_16bits, rand_16bits)
                    self.rand_key_dict.insert(key, rand_16bits)
                    hash_bool = False
            count += 1
            if count % 100 == 0:
                print(f"count: {count}")
        dicted_tuple = self.rand_key_dict.retrieve(key)
        self.rand_key_dict.insert(dicted_tuple[0], dicted_tuple[1])  # could do default value arguments to also accept a tuple directly
        return dicted_tuple[1]  # % len(self.hash_table)  ### with length of 1, all 0

iterations = 25
probing_functions_list = ['linear', 'quadratic', 'double', 'prime', '3/2', "to Euler's number", 'cubic', 'exponential']  #, 'rand double hashing']
collisions_data = [HashTable(1) for _ in probing_functions_list]

index_keys = [n for n in range(1001)]
# print('keys complete')
for _ in range(iterations):
    print(f"iteration {_}")
    # for N in range(0, 10000, 1000)[0:3]:
    for N in list(range(0, 700, 125)) + [1000]:
        exp_hash_table = ExperimentalHashTable(1, 'sep')  # provides hash vales for key for each iteration * each N, which are the same for each probing function, per N, per iteration
        # print(f"commencing key data:")
        key_data = [exp_hash_table.rand_hash(key) for key in index_keys[0:N]]
        # print('key data complete')
        # print(key_data)  # consider plotting to show randomness
        for probing_function in probing_functions_list:
            # print(f"probing function: {probing_function}")
            hash_table = HashTable(2, probing_function)  # start at 2 to avoid integer modulo by 0
            for key in key_data:
                hash_table.insert(key, '')
            collisions_data[probing_functions_list.index(probing_function)].insert(N, hash_table.collision_count)  # hash table does not update values, maybe make update method
            # print(f"{probing_function} completed for N: {N}")
        
# Idea: cluster hash table - a (HashTable): assuming keys are deliberately colliding, provide statistics per key cluster  ## some of what we wonder about in science is the validity of a given hash function from observations to ... or from theoretical to observations ... or ... 
plt.figure()            # combine multiple hash functions -> 'good' clustering (and/or Monte Carlo the clusters ?)
for label, each in zip(probing_functions_list, collisions_data):
    all_tuples = each.get_all()
    # print(all_tuples)
    # all_tuples.sort(key= lambda p: (-p[0], -p[1]))
    # plt.plot([each_tuple[0] for each_tuple in all_tuples], [each_tuple[1] for each_tuple in all_tuples], label=label)
    # plt.scatter([each_tuple[0] for each_tuple in all_tuples], [each_tuple[1] for each_tuple in all_tuples], label=label)  # scatter data looks like the average would look pretty clean
    data_dict = HashTable(2)
    for each_tuple in all_tuples:
        if not data_dict.contains(each_tuple[0]):
            data_dict.insert(each_tuple[0], 0)
        running_sum_tuple = data_dict.retrieve(each_tuple[0])
        data_dict.insert(each_tuple[0], running_sum_tuple[1] + each_tuple[1])
    plt.scatter([each_tuple[0] for each_tuple in data_dict.get_all()], [each_tuple[1] for each_tuple in data_dict.get_all()], label=label, marker='+')
plt.legend()
plt.show()
# user_data_sets = {}
# with open('user_data_csv.txt', 'r') as open_file:
#     csv_reader = csv.reader(open_file)
#     for row in csv_reader:
#         user = row['user_id']
#         game = row['item_id']
#         if user not in user_data_sets:
#             user_data_sets[user] = set()
#         user_data_sets[user].add(game)

# standard Venn diagrams only go up to 

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7


In [3]:
class OrderedSet(list):
    def __iadd__(self, value):
        return super().__iadd__(value) if self.count(value) == 0 else None